In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import logging
import os
log = logging.getLogger('Exploratory_Data_Analysis')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s : %(message)s', datefmt='%H:%M:%S')

In [2]:
if os.path.exists('../data/bank_transactions_data_2.csv'):
        df = pd.read_csv('../data/bank_transactions_data_2.csv')
        log.info('Data has successfully been loaded')
else:
    log.error('File Not Found')

11:19:09 - INFO : Data has successfully been loaded


In [3]:
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [4]:
df['amount_to_balance_ratio'] = (df['TransactionAmount'] / df['AccountBalance'] )

In [5]:
df['avg_txn_amount_account'] = df.groupby('AccountID')['TransactionAmount'].transform('mean')

In [6]:
df['txn_amount_account'] = df.groupby('AccountID')['TransactionAmount'].transform('count')

In [7]:
user_stats = df.groupby('AccountID')['TransactionAmount'].agg(['mean','std']).reset_index()
df = df.merge(user_stats, on='AccountID', how='left', suffixes=('','_user'))
df['amount_zscore'] = (df['TransactionAmount'] - df['mean']) / df['std']

In [8]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])

In [9]:
df['diff_btwn_txn_times'] = abs(df['TransactionDate'] - df['PreviousTransactionDate']).dt.days
df['avg_time_btwn_txns'] = df.groupby('AccountID')['diff_btwn_txn_times'].transform('mean')

In [10]:
df['amount_to_user_avg'] = df['TransactionAmount'] / df['avg_txn_amount_account']

In [11]:
df['unique_merchant_user'] = df.groupby('AccountID')['MerchantID'].transform('nunique')

In [12]:
df['TransactionHour'] = df['TransactionDate'].dt.hour

In [13]:
df['TransactionDayOfTheWeek'] = df['TransactionDate'].dt.dayofweek

In [14]:
df['is_night'] = df['TransactionHour'].apply(lambda h: 1 if h < 6 or h > 22 else 0)

In [15]:
df['unique_devices_per_account'] = df.groupby('AccountID')['DeviceID'].transform('count')

In [16]:
df['unique_ips_per_account'] = df.groupby('AccountID')['IP Address'].transform('count')

In [17]:
df['unique_accounts_per_devices'] = df.groupby('DeviceID')['AccountID'].transform('count')

In [18]:
df['avg_loginAttempts'] = df.groupby('AccountID')['LoginAttempts'].transform('mean')

In [19]:
df['merchant_avg_amount'] = df.groupby('MerchantID')['TransactionAmount'].transform('mean')

In [20]:
df['merchant_amount_deviation'] = (df['TransactionAmount'] - df['merchant_avg_amount'])

In [21]:
df['loginAttemps_excess'] = df['LoginAttempts'] - df['avg_loginAttempts']

In [22]:
df['TransactionAmount'] = np.log1p(df['TransactionAmount'])

In [23]:
df.sort_values(by='AccountID')

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,...,TransactionHour,TransactionDayOfTheWeek,is_night,unique_devices_per_account,unique_ips_per_account,unique_accounts_per_devices,avg_loginAttempts,merchant_avg_amount,merchant_amount_deviation,loginAttemps_excess
1312,TX001313,AC00001,3.887525,2023-09-15 17:00:20,Debit,Denver,D000649,59.12.96.11,M034,Branch,...,17,4,0,2,2,8,1.000000,360.085238,-312.295238,0.000000
2016,TX002017,AC00001,5.365836,2023-11-14 16:56:34,Debit,Atlanta,D000492,45.241.13.208,M003,Online,...,16,1,0,2,2,1,1.000000,214.016923,-1.046923,0.000000
1598,TX001599,AC00002,5.981818,2023-07-24 16:14:05,Debit,Milwaukee,D000269,165.38.56.205,M059,Online,...,16,0,0,7,7,6,1.142857,256.051111,139.108889,-0.142857
1673,TX001674,AC00002,5.807121,2023-09-11 17:52:59,Debit,Las Vegas,D000041,1.198.76.182,M040,ATM,...,17,0,0,7,7,4,1.142857,291.160000,40.500000,-0.142857
1028,TX001029,AC00002,6.248952,2023-12-21 17:00:50,Credit,Charlotte,D000420,59.132.247.252,M026,ATM,...,17,3,0,7,7,4,1.142857,308.114444,208.355556,-0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,TX001943,AC00499,6.731364,2023-11-27 16:14:25,Debit,Phoenix,D000189,217.233.219.196,M002,Branch,...,16,0,0,7,7,2,1.000000,290.427097,546.862903,0.000000
2349,TX002350,AC00500,4.909341,2023-10-09 16:44:50,Debit,San Jose,D000219,100.137.90.188,M026,Online,...,16,0,0,4,4,4,1.000000,308.114444,-173.564444,0.000000
2374,TX002375,AC00500,4.872292,2023-07-06 16:47:00,Credit,San Jose,D000179,37.143.57.20,M070,ATM,...,16,3,0,4,4,4,1.000000,261.320625,-131.700625,0.000000
1715,TX001716,AC00500,5.424642,2023-03-13 17:57:47,Credit,Omaha,D000142,147.114.91.132,M097,Branch,...,17,0,0,4,4,9,1.000000,300.961154,-75.031154,0.000000
